In [1]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pandas as pd
df1 = pd.read_csv(r"D:\desktop\B\数据\标准化后数据\df1.csv")
df2 = pd.read_excel(r"D:\desktop\B\数据\标准化后数据\df2.xlsx")
df1[['mort', 'CONS','MV','CRRT']] = df1[['mort',  'CONS','MV','CRRT']].astype('category')
df2[['mort',  'CONS','MV','CRRT']] = df2[['mort',  'CONS','MV','CRRT']].astype('category')
selected_features = ["CONS", "LDH", "MV", "AST", "CRRT", "U", "L", "HR", "D.Dimer", "CR", "age", "ALT"]
target = 'mort'
X = df1[selected_features]
y = df1[target]
X_train, X_internal_test, y_train, y_internal_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_external_test = df2[selected_features]
y_external_test = df2[target]

In [2]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils import resample

RANDOM_SEED = 42

# 定义计算置信区间的函数
def bootstrap_ci(y_true, y_pred, metric_func, n_bootstraps=10000, ci=95, random_state=42):
    rng = np.random.RandomState(random_state)
    bootstrapped_scores = []
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    for _ in range(n_bootstraps):
        indices = resample(np.arange(len(y_true)), replace=True, random_state=rng)
        score = metric_func(y_true[indices], y_pred[indices])
        bootstrapped_scores.append(score)
    
    lower_bound = np.percentile(bootstrapped_scores, (100-ci)/2)
    upper_bound = np.percentile(bootstrapped_scores, 100-(100-ci)/2)
    return lower_bound, upper_bound

params_svm = {
    'C': [0.5, 1, 2, 5],
    'kernel': ['poly', 'rbf'],
    'degree': [2, 3, 4]
}

best_models_svm = {}

for i in range(1, len(selected_features) + 1):
    current_features = selected_features[:i]
    X_train_current = X_train[current_features]
    X_internal_test_current = X_internal_test[current_features]
    X_external_test_current = X_external_test[current_features]
    
    model_svm = SVC(probability=True, random_state=RANDOM_SEED)
    grid_svm = GridSearchCV(model_svm, params_svm, cv=10, scoring='roc_auc', n_jobs=-1)
    grid_svm.fit(X_train_current, y_train)
    
    best_model_svm = grid_svm.best_estimator_

    # 使用交叉验证在训练集和内部验证集上进行预测
    y_train_pred_cv = cross_val_predict(best_model_svm, X_train_current, y_train, cv=10, method='predict')
    y_train_proba_cv = cross_val_predict(best_model_svm, X_train_current, y_train, cv=10, method='predict_proba')[:, 1]
    
    y_internal_pred_cv = cross_val_predict(best_model_svm, X_internal_test_current, y_internal_test, cv=10, method='predict')
    y_internal_proba_cv = cross_val_predict(best_model_svm, X_internal_test_current, y_internal_test, cv=10, method='predict_proba')[:, 1]

    # 在外部验证集上进行预测
    y_external_pred_svm = best_model_svm.predict(X_external_test_current)
    y_external_proba_svm = best_model_svm.predict_proba(X_external_test_current)[:, 1]

    # 计算训练集的各项指标分数
    train_accuracy = accuracy_score(y_train, y_train_pred_cv)
    train_precision = precision_score(y_train, y_train_pred_cv, average='weighted', zero_division=0)
    train_recall = recall_score(y_train, y_train_pred_cv, average='weighted')
    train_f1 = f1_score(y_train, y_train_pred_cv, average='weighted')
    train_auc = roc_auc_score(y_train, y_train_proba_cv)

    # 计算训练集的各项指标的置信区间
    train_ci = {
        'accuracy_ci': bootstrap_ci(y_train, y_train_pred_cv, accuracy_score),
        'precision_ci': bootstrap_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall_ci': bootstrap_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
        'f1_score_ci': bootstrap_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
        'auc_ci': bootstrap_ci(y_train, y_train_proba_cv, roc_auc_score)
    }

    # 计算内部验证集的各项指标分数
    internal_accuracy = accuracy_score(y_internal_test, y_internal_pred_cv)
    internal_precision = precision_score(y_internal_test, y_internal_pred_cv, average='weighted', zero_division=0)
    internal_recall = recall_score(y_internal_test, y_internal_pred_cv, average='weighted')
    internal_f1 = f1_score(y_internal_test, y_internal_pred_cv, average='weighted')
    internal_auc = roc_auc_score(y_internal_test, y_internal_proba_cv)

    # 计算内部验证集的各项指标的置信区间
    internal_ci = {
        'accuracy_ci': bootstrap_ci(y_internal_test, y_internal_pred_cv, accuracy_score),
        'precision_ci': bootstrap_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall_ci': bootstrap_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
        'f1_score_ci': bootstrap_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
        'auc_ci': bootstrap_ci(y_internal_test, y_internal_proba_cv, roc_auc_score)
    }

    # 计算外部验证集的各项指标分数
    external_accuracy = accuracy_score(y_external_test, y_external_pred_svm)
    external_precision = precision_score(y_external_test, y_external_pred_svm, average='weighted', zero_division=0)
    external_recall = recall_score(y_external_test, y_external_pred_svm, average='weighted')
    external_f1 = f1_score(y_external_test, y_external_pred_svm, average='weighted')
    external_auc = roc_auc_score(y_external_test, y_external_proba_svm)

    # 计算外部验证集的各项指标的置信区间
    external_ci = {
        'accuracy_ci': bootstrap_ci(y_external_test, y_external_pred_svm, accuracy_score),
        'precision_ci': bootstrap_ci(y_external_test, y_external_pred_svm, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall_ci': bootstrap_ci(y_external_test, y_external_pred_svm, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
        'f1_score_ci': bootstrap_ci(y_external_test, y_external_pred_svm, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
        'auc_ci': bootstrap_ci(y_external_test, y_external_proba_svm, roc_auc_score)
    }

    best_models_svm[i] = {
        'features': current_features,
        'train_metrics': {
            'accuracy': train_accuracy,
            'precision': train_precision,
            'recall': train_recall,
            'f1_score': train_f1,
            'auc': train_auc
        },
        'train_ci': train_ci,
        'internal_metrics': {
            'accuracy': internal_accuracy,
            'precision': internal_precision,
            'recall': internal_recall,
            'f1_score': internal_f1,
            'auc': internal_auc
        },
        'internal_ci': internal_ci,
        'external_metrics': {
            'accuracy': external_accuracy,
            'precision': external_precision,
            'recall': external_recall,
            'f1_score': external_f1,
            'auc': external_auc
        },
        'external_ci': external_ci,
        'best_params': grid_svm.best_params_
    }

# 输出每个模型的结果
for i, results in best_models_svm.items():
    print(f"\nModel with top {i} features: {results['features']}")
    print("Best Parameters:", results['best_params'])

    print("Training Set Metrics:")
    for metric, value in results['train_metrics'].items():
        print(f"{metric.capitalize()}: {value:.4f} 置信区间: {results['train_ci'][metric+'_ci'][0]:.4f}-{results['train_ci'][metric+'_ci'][1]:.4f}")

    print("Internal Validation Set Metrics:")
    for metric, value in results['internal_metrics'].items():
        print(f"{metric.capitalize()}: {value:.4f} 置信区间: {results['internal_ci'][metric+'_ci'][0]:.4f}-{results['internal_ci'][metric+'_ci'][1]:.4f}")
    
    print("External Validation Set Metrics:")
    for metric, value in results['external_metrics'].items():
        print(f"{metric.capitalize()}: {value:.4f} 置信区间: {results['external_ci'][metric+'_ci'][0]:.4f}-{results['external_ci'][metric+'_ci'][1]:.4f}")



Model with top 1 features: ['CONS']
Best Parameters: {'C': 0.5, 'degree': 2, 'kernel': 'poly'}
Training Set Metrics:
Accuracy: 0.8676 置信区间: 0.8186-0.9118
Precision: 0.8760 置信区间: 0.8312-0.9185
Recall: 0.8676 置信区间: 0.8186-0.9118
F1_score: 0.8706 置信区间: 0.8235-0.9134
Auc: 0.7913 置信区间: 0.7019-0.8720
Internal Validation Set Metrics:
Accuracy: 0.7045 置信区间: 0.6136-0.7955
Precision: 0.7292 置信区间: 0.6319-0.8282
Recall: 0.7045 置信区间: 0.6136-0.7955
F1_score: 0.7137 置信区间: 0.6200-0.8019
Auc: 0.6699 置信区间: 0.5373-0.7944
External Validation Set Metrics:
Accuracy: 0.8558 置信区间: 0.7885-0.9231
Precision: 0.8503 置信区间: 0.7767-0.9215
Recall: 0.8558 置信区间: 0.7885-0.9231
F1_score: 0.8476 置信区间: 0.7696-0.9183
Auc: 0.7628 置信区间: 0.6643-0.8610

Model with top 2 features: ['CONS', 'LDH']
Best Parameters: {'C': 0.5, 'degree': 4, 'kernel': 'poly'}
Training Set Metrics:
Accuracy: 0.8578 置信区间: 0.8088-0.9020
Precision: 0.8569 置信区间: 0.8060-0.9057
Recall: 0.8578 置信区间: 0.8088-0.9020
F1_score: 0.8573 置信区间: 0.8059-0.9037
Auc: 0.

In [3]:
import pandas as pd
import os

# 设置输出文件夹路径
output_folder = r"D:\desktop\B\结果\表格\内部外部验证结果"
os.makedirs(output_folder, exist_ok=True)  # 确保输出文件夹存在

# 用于存储结果的列表
output_data = []

# 遍历模型结果并将其组织为字典
for i, results in best_models_svm.items():
    model_name = f"Model with top {i} features"
    for metric in results['train_metrics'].keys():
        output_data.append({
            'Model': model_name,
            'Dataset': 'Training Set',
            'Metric': metric.capitalize(),
            'Value': results['train_metrics'][metric],
            'Confidence Interval': f"{results['train_ci'][metric+'_ci'][0]:.4f}-{results['train_ci'][metric+'_ci'][1]:.4f}"
        })
        output_data.append({
            'Model': model_name,
            'Dataset': 'Internal Validation Set',
            'Metric': metric.capitalize(),
            'Value': results['internal_metrics'][metric],
            'Confidence Interval': f"{results['internal_ci'][metric+'_ci'][0]:.4f}-{results['internal_ci'][metric+'_ci'][1]:.4f}"
        })
        output_data.append({
            'Model': model_name,
            'Dataset': 'External Validation Set',
            'Metric': metric.capitalize(),
            'Value': results['external_metrics'][metric],
            'Confidence Interval': f"{results['external_ci'][metric+'_ci'][0]:.4f}-{results['external_ci'][metric+'_ci'][1]:.4f}"
        })

# 将列表转换为 DataFrame
output_df = pd.DataFrame(output_data)

# 设置文件名
output_file = os.path.join(output_folder, "SVM_Results.xlsx")

# 将 DataFrame 保存为 Excel 文件
output_df.to_excel(output_file, index=False)

print(f"结果已保存至：{output_file}")


结果已保存至：D:\desktop\B\结果\表格\内部外部验证结果\SVM_Results.xlsx


11个特征单独训练

In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_predict

RANDOM_SEED = 42

# 使用所有选定的最佳特征
selected_features = ['CONS', 'LDH', 'MV', 'AST', 'CRRT', 'U', 'L', 'HR', 'D.Dimer', 'CR', 'age']

# 使用最佳参数
best_params = {'C': 1, 'degree': 2, 'kernel': 'rbf'}

# 创建SVM模型实例
model_svm = SVC(probability=True, random_state=RANDOM_SEED, **best_params)

# 训练模型
model_svm.fit(X_train[selected_features], y_train)

# 进行交叉验证预测（训练集和内部验证集）
y_train_pred_cv = cross_val_predict(model_svm, X_train[selected_features], y_train, cv=10, method='predict')
y_train_proba_cv = cross_val_predict(model_svm, X_train[selected_features], y_train, cv=10, method='predict_proba')[:, 1]

y_internal_pred_cv = cross_val_predict(model_svm, X_internal_test[selected_features], y_internal_test, cv=10, method='predict')
y_internal_proba_cv = cross_val_predict(model_svm, X_internal_test[selected_features], y_internal_test, cv=10, method='predict_proba')[:, 1]

# 对外部验证集进行单次预测
y_external_pred_svm = model_svm.predict(X_external_test[selected_features])
y_external_proba_svm = model_svm.predict_proba(X_external_test[selected_features])[:, 1]

# 输出训练集、内部验证集和外部验证集的指标
train_accuracy = accuracy_score(y_train, y_train_pred_cv)
train_precision = precision_score(y_train, y_train_pred_cv, average='weighted', zero_division=0)
train_recall = recall_score(y_train, y_train_pred_cv, average='weighted')
train_f1 = f1_score(y_train, y_train_pred_cv, average='weighted')
train_auc = roc_auc_score(y_train, y_train_proba_cv)

internal_accuracy = accuracy_score(y_internal_test, y_internal_pred_cv)
internal_precision = precision_score(y_internal_test, y_internal_pred_cv, average='weighted', zero_division=0)
internal_recall = recall_score(y_internal_test, y_internal_pred_cv, average='weighted')
internal_f1 = f1_score(y_internal_test, y_internal_pred_cv, average='weighted')
internal_auc = roc_auc_score(y_internal_test, y_internal_proba_cv)

external_accuracy = accuracy_score(y_external_test, y_external_pred_svm)
external_precision = precision_score(y_external_test, y_external_pred_svm, average='weighted', zero_division=0)
external_recall = recall_score(y_external_test, y_external_pred_svm, average='weighted')
external_f1 = f1_score(y_external_test, y_external_pred_svm, average='weighted')
external_auc = roc_auc_score(y_external_test, y_external_proba_svm)

# 输出结果
print(f"Model with top {len(selected_features)} features: {selected_features}")
print("Best Parameters:", best_params)

print("Training Set Metrics:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"Precision: {train_precision:.4f}")
print(f"Recall: {train_recall:.4f}")
print(f"F1 Score: {train_f1:.4f}")
print(f"AUC: {train_auc:.4f}")

print("\nInternal Validation Set Metrics:")
print(f"Accuracy: {internal_accuracy:.4f}")
print(f"Precision: {internal_precision:.4f}")
print(f"Recall: {internal_recall:.4f}")
print(f"F1 Score: {internal_f1:.4f}")
print(f"AUC: {internal_auc:.4f}")

print("\nExternal Validation Set Metrics:")
print(f"Accuracy: {external_accuracy:.4f}")
print(f"Precision: {external_precision:.4f}")
print(f"Recall: {external_recall:.4f}")
print(f"F1 Score: {external_f1:.4f}")
print(f"AUC: {external_auc:.4f}")


Model with top 11 features: ['CONS', 'LDH', 'MV', 'AST', 'CRRT', 'U', 'L', 'HR', 'D.Dimer', 'CR', 'age']
Best Parameters: {'C': 1, 'degree': 2, 'kernel': 'rbf'}
Training Set Metrics:
Accuracy: 0.8922
Precision: 0.8922
Recall: 0.8922
F1 Score: 0.8922
AUC: 0.9083

Internal Validation Set Metrics:
Accuracy: 0.8523
Precision: 0.8482
Recall: 0.8523
F1 Score: 0.8490
AUC: 0.9121

External Validation Set Metrics:
Accuracy: 0.9038
Precision: 0.9036
Recall: 0.9038
F1 Score: 0.8994
AUC: 0.8905


In [3]:
import os

# 指定保存路径
save_dir = r'D:\desktop\B\结果\表格\SVM真实标签和预测概率'

# 检查文件夹是否存在，如果不存在就创建
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 获取训练集的真实标签和预测概率
y_train_true = y_train.values
train_data = pd.DataFrame({
    'True_Label': y_train_true,
    'Predicted_Probability': y_train_proba_cv
})
train_data.to_csv(os.path.join(save_dir, 'train_labels_and_probabilities.csv'), index=False)

# 获取内部验证集的真实标签和预测概率
y_internal_true = y_internal_test.values
internal_data = pd.DataFrame({
    'True_Label': y_internal_true,
    'Predicted_Probability': y_internal_proba_cv
})
internal_data.to_csv(os.path.join(save_dir, 'internal_validation_labels_and_probabilities.csv'), index=False)

# 获取外部验证集的真实标签和预测概率
y_external_true = y_external_test.values
external_data = pd.DataFrame({
    'True_Label': y_external_true,
    'Predicted_Probability': y_external_proba_svm
})
external_data.to_csv(os.path.join(save_dir, 'external_validation_labels_and_probabilities.csv'), index=False)

print("训练集、内部验证集和外部验证集的标签和预测概率已保存到指定文件夹。")


训练集、内部验证集和外部验证集的标签和预测概率已保存到指定文件夹。


In [3]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_predict

RANDOM_SEED = 42

# 使用所有选定的最佳特征
selected_features = ['CONS', 'LDH', 'MV', 'AST', 'CRRT', 'U', 'L']

# 使用最佳参数
best_params = {'C': 0.5, 'degree': 2, 'kernel': 'rbf'}

# 创建SVM模型实例
model_svm = SVC(probability=True, random_state=RANDOM_SEED, **best_params)

# 训练模型
model_svm.fit(X_train[selected_features], y_train)

# 进行交叉验证预测（训练集和内部验证集）
y_train_pred_cv = cross_val_predict(model_svm, X_train[selected_features], y_train, cv=10, method='predict')
y_train_proba_cv = cross_val_predict(model_svm, X_train[selected_features], y_train, cv=10, method='predict_proba')[:, 1]

y_internal_pred_cv = cross_val_predict(model_svm, X_internal_test[selected_features], y_internal_test, cv=10, method='predict')
y_internal_proba_cv = cross_val_predict(model_svm, X_internal_test[selected_features], y_internal_test, cv=10, method='predict_proba')[:, 1]

# 对外部验证集进行单次预测
y_external_pred_svm = model_svm.predict(X_external_test[selected_features])
y_external_proba_svm = model_svm.predict_proba(X_external_test[selected_features])[:, 1]

# 输出训练集、内部验证集和外部验证集的指标
train_accuracy = accuracy_score(y_train, y_train_pred_cv)
train_precision = precision_score(y_train, y_train_pred_cv, average='weighted', zero_division=0)
train_recall = recall_score(y_train, y_train_pred_cv, average='weighted')
train_f1 = f1_score(y_train, y_train_pred_cv, average='weighted')
train_auc = roc_auc_score(y_train, y_train_proba_cv)

internal_accuracy = accuracy_score(y_internal_test, y_internal_pred_cv)
internal_precision = precision_score(y_internal_test, y_internal_pred_cv, average='weighted', zero_division=0)
internal_recall = recall_score(y_internal_test, y_internal_pred_cv, average='weighted')
internal_f1 = f1_score(y_internal_test, y_internal_pred_cv, average='weighted')
internal_auc = roc_auc_score(y_internal_test, y_internal_proba_cv)

external_accuracy = accuracy_score(y_external_test, y_external_pred_svm)
external_precision = precision_score(y_external_test, y_external_pred_svm, average='weighted', zero_division=0)
external_recall = recall_score(y_external_test, y_external_pred_svm, average='weighted')
external_f1 = f1_score(y_external_test, y_external_pred_svm, average='weighted')
external_auc = roc_auc_score(y_external_test, y_external_proba_svm)

# 输出结果
print(f"Model with top {len(selected_features)} features: {selected_features}")
print("Best Parameters:", best_params)

print("Training Set Metrics:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"Precision: {train_precision:.4f}")
print(f"Recall: {train_recall:.4f}")
print(f"F1 Score: {train_f1:.4f}")
print(f"AUC: {train_auc:.4f}")

print("\nInternal Validation Set Metrics:")
print(f"Accuracy: {internal_accuracy:.4f}")
print(f"Precision: {internal_precision:.4f}")
print(f"Recall: {internal_recall:.4f}")
print(f"F1 Score: {internal_f1:.4f}")
print(f"AUC: {internal_auc:.4f}")

print("\nExternal Validation Set Metrics:")
print(f"Accuracy: {external_accuracy:.4f}")
print(f"Precision: {external_precision:.4f}")
print(f"Recall: {external_recall:.4f}")
print(f"F1 Score: {external_f1:.4f}")
print(f"AUC: {external_auc:.4f}")


Model with top 7 features: ['CONS', 'LDH', 'MV', 'AST', 'CRRT', 'U', 'L']
Best Parameters: {'C': 0.5, 'degree': 2, 'kernel': 'rbf'}
Training Set Metrics:
Accuracy: 0.8824
Precision: 0.8808
Recall: 0.8824
F1 Score: 0.8815
AUC: 0.9100

Internal Validation Set Metrics:
Accuracy: 0.7955
Precision: 0.7883
Recall: 0.7955
F1 Score: 0.7701
AUC: 0.8613

External Validation Set Metrics:
Accuracy: 0.8654
Precision: 0.8624
Recall: 0.8654
F1 Score: 0.8564
AUC: 0.9063


In [4]:
import os

# 指定保存路径
save_dir = r'D:\desktop\B\结果\表格\SVM真实标签和预测概率(7)'

# 检查文件夹是否存在，如果不存在就创建
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 获取训练集的真实标签和预测概率
y_train_true = y_train.values
train_data = pd.DataFrame({
    'True_Label': y_train_true,
    'Predicted_Probability': y_train_proba_cv
})
train_data.to_csv(os.path.join(save_dir, 'train_labels_and_probabilities.csv'), index=False)

# 获取内部验证集的真实标签和预测概率
y_internal_true = y_internal_test.values
internal_data = pd.DataFrame({
    'True_Label': y_internal_true,
    'Predicted_Probability': y_internal_proba_cv
})
internal_data.to_csv(os.path.join(save_dir, 'internal_validation_labels_and_probabilities.csv'), index=False)

# 获取外部验证集的真实标签和预测概率
y_external_true = y_external_test.values
external_data = pd.DataFrame({
    'True_Label': y_external_true,
    'Predicted_Probability': y_external_proba_svm
})
external_data.to_csv(os.path.join(save_dir, 'external_validation_labels_and_probabilities.csv'), index=False)

print("训练集、内部验证集和外部验证集的标签和预测概率已保存到指定文件夹。")


训练集、内部验证集和外部验证集的标签和预测概率已保存到指定文件夹。
